In [2]:
import numpy as np
import cv2
import tensorflow as tf

In [3]:
face_model = cv2.CascadeClassifier('./haarcascades/haarcascade_frontalface_default.xml')

In [4]:
model = tf.keras.models.load_model("./masknet.h5")

In [5]:
CLASSES = {
    0: "Mask",
    1: "No Mask"
}

In [6]:
vid  = cv2.VideoCapture(0)
while True:
      ret, frame = vid.read()
      img = frame
      # img = cv2.cvtColor(frame, cv2.IMREAD_GRAYSCALE)

      faces = face_model.detectMultiScale(img,scaleFactor=1.1, minNeighbors=4) #returns a list of (x,y,w,h) tuples

      for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),1)
      
      cv2.imshow("frame", img)

      if cv2.waitKey(1) & 0xFF == ord('q'):
        break
      
cv2.destroyAllWindows()

In [10]:
vid  = cv2.VideoCapture(0)
while True:
      ret, frame = vid.read()
      img = cv2.cvtColor(frame, cv2.IMREAD_GRAYSCALE)

      faces = face_model.detectMultiScale(img,scaleFactor=1.1, minNeighbors=4) #returns a list of (x,y,w,h) tuples

      for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),1)

            label = [0 for i in range(len(faces))]
            new_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #colored output image
            for i in range(len(faces)):
                  (x,y,w,h) = faces[i]
                  crop = new_img[y:y+h,x:x+w]
                  crop = cv2.resize(crop,(128,128))
                  crop = np.reshape(crop,[1,128,128,3])/255.0
                  mask_result = model.predict(crop)
                  class_index = np.argmax(mask_result[0])
                  result = CLASSES[class_index]
                  cv2.putText(img, str(round(mask_result[0][class_index], 2)), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 
                              1, (0, 0, 255), 2)
                  cv2.putText(img, result, (x+20,y+20), cv2.FONT_HERSHEY_SIMPLEX, 
                              1, (0, 0, 255), 2)
      
      cv2.imshow("frame", img)

      if cv2.waitKey(1) & 0xFF == ord('q'):
        break
      
cv2.destroyAllWindows()
